In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.utils import to_categorical
import os
import time
loc = os.getcwd()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
config =  tf.compat.v1.ConfigProto()
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config=config)
os.environ['TF_CUDNN_WORKSPACE_LIMIT_IN_MB'] = '512'

In [ ]:
class Generator(tf.keras.utils.Sequence):

    def __init__(self, x, classes, batchsize, steps):
        self.x = x
        self.classes = classes
        self.batchsize = batchsize
        self.n_index = np.unique(np.array(classes))
        self.steps = steps
        self.temp = 35

    def __len__(self):
        return self.steps

    def read_img(self, path):

        img = cv2.resize(cv2.imread(path), (224, 224), interpolation=cv2.INTER_CUBIC)
        img = img[self.temp:-self.temp, self.temp:-self.temp, :]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) / 255

        return img

    def __getitem__(self, idx):
        inp = [np.zeros([self.batchsize, 154, 154, 1]) for _ in range(2)]
        out = np.zeros(self.batchsize, dtype=np.int32)
        indices = np.random.choice(list(range(self.batchsize)), size=self.batchsize // 2, replace=False)
        out[indices] = 1

        cond = 1
        while cond:
            cat = np.random.choice(self.n_index, size=[self.batchsize * 2], replace=False)
            index = [list(np.where(np.array(self.classes) == i)[0]) for i in cat]
            for i in range(self.batchsize):
                if len(index[i]) <= 1:
                    cond = 1
                    break
                else:
                    cond = 0

        j = self.batchsize
        pos = list(range(self.batchsize))
        np.random.shuffle(pos)

        for i in range(self.batchsize):
            index1 = np.random.choice(index[i], size=[2], replace=False)
            inp[0][i, :, :, 0] = self.read_img(self.x[index1[0]])

            if out[i] == 1:
                inp[1][i, :, :, 0] = self.read_img(self.x[index1[-1]])

            else:
                index2 = np.random.choice(index[j])
                inp[1][i, :, :, 0] = self.read_img(self.x[index2])
                j += 1

        return inp, out

In [ ]:
def get_n_way_oneshot(x, classes, m, n):
    n_classes = np.unique(np.array(classes))
    
    # In the dataset there are certain classes that has only one sample in it.
    # To avoid taking those samples we perform a check that the len of the indexes 
    # are greater than two or not.
    
    temp = 35
    cond = 1
    while cond:
        inp_c = np.random.choice(n_classes)
        inp_class = np.where(classes == inp_c)[0]
        if len(inp_class) <= 1:
            cond = 1
            
        else:
            cond = 0
            
    inp_n = np.random.choice(inp_class, size=2, replace=False)
    print(f"Input Class : {inp_c}")
    print(f"Input Files : {inp_n}")
                
    img = cv2.cvtColor(cv2.imread(x[inp_n[0]]), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
    img = img[temp:-temp, temp:-temp, :]
    inp = img / 255
    
    test = np.zeros((m, n, 154, 154, 3), dtype=np.float32)
    index_i = np.random.randint(0, m)
    index_j = np.random.randint(0, n)
    
    # Conditions to make sure that the input class is not duplicated in the test set
    # We perform a check to remove such errors.
    
    cond = 1
    while cond:
        cat = np.random.choice(n_classes, size=[m, n], replace=False)
        if inp_n.any() in np.ndarray.flatten(cat):
            pass
        
        else:
            cond = 0
        
    print(f"Test Files : {np.ndarray.flatten(cat)}")
    w = 154
    print(index_i, index_j)
    for i in range(m):
        for j in range(n):
            if index_i == i and index_j == j:
                
                img = cv2.cvtColor(cv2.imread(x[inp_n[1]]), cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (224, 224),  interpolation = cv2.INTER_CUBIC)
                img = img[temp:-temp, temp:-temp, :]
                test[i, j, :, :, :] = img / 255
                
            else:
                
                index2 = np.random.choice(np.where(classes == cat[i][j])[0])
                img = cv2.cvtColor(cv2.imread(x[index2]), cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
                img = img[temp:-temp, temp:-temp, :]
                test[i, j, :, :, :] = img / 255
                
    inp = np.expand_dims(inp, axis=0)
    plott = np.zeros([w*m, w*n, 3])
    
    for i in range(m):
        for j in range(n):
            
            plott[i*w:(i+1)*w, j*w:(j+1)*w, :] = test[i, j, :, :, :]

    fig, ax = plt.subplots(1, 2, figsize=(17, 17))
    ax[0].imshow(inp[0, :, :, :])
    ax[0].axis("off")

    ax[1].imshow(plott[:, :, :])
    ax[1].axis("off")
    plt.show()
                
    return inp, test

In [ ]:
def euclid_distance(inp):

    eps = 1e-09
    tensor1, tensor2 = inp
    K = tf.keras.backend
    dist = K.sqrt(K.maximum(K.sum(K.square(tensor1 - tensor2), axis=-1, keepdims=True), eps))

    return dist


def get_output_shape(shape):

    return shape[0], 1

In [ ]:
def siamese_model():

    weight_decay = 1e-03
    shape = [154, 154, 3]
    reg = tf.keras.regularizers.l2(weight_decay)

    l_input = tf.keras.layers.Input(shape=shape)
    r_input = tf.keras.layers.Input(shape=shape)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same", kernel_regularizer=reg, use_bias=False, input_shape=tuple(shape)))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D())

    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", kernel_regularizer=reg, use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D())

    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", kernel_regularizer=reg, use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D())

    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", kernel_regularizer=reg, use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D())

    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", kernel_regularizer=reg, use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D())

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32, kernel_regularizer=reg))

    l_encoder = model(l_input)
    r_encoder = model(r_input)

    L1_layer = tf.keras.layers.Lambda(euclid_distance, output_shape=get_output_shape)([l_encoder, r_encoder])
    prediction = tf.keras.layers.Dense(1, activation='sigmoid')(L1_layer)

    model = tf.keras.models.Model(inputs=[l_input, r_input], outputs=prediction)

    return model

In [ ]:
# For Testing Purposes

train_file_list = []
valid_file_list = []

path = r"list_eval_partition.csv"
data = pd.read_csv(path)

file_path = r"path to images"
valid_persons = []

for i in range(len(data["Person"])):
    
    if data["partition"][i] == 1:
        valid_file_list.append(os.path.join(file_path, data["image_id"][i]))
        valid_persons.append(data["Person"][i])

model = siamese_model()
lr = 1e-05
opt = tf.keras.optimizers.RMSprop(learning_rate=lr, momentum=0.9)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
model.load_weights("siamese_valid.h5")

print(model.summary())

In [ ]:
m = 3
n = 3
x, y = get_n_way_oneshot(valid_file_list, valid_persons, m, n)

y = np.reshape(y, newshape=[m*n, 154, 154, 3])
pred = []
for i in range(m*n):
    inp = [x, np.expand_dims(y[i], axis=0)]
    pred.append(model.predict(inp))
    
pred = np.array(pred, dtype=np.float64)
ind = np.argmax(pred)
print(f"Image No : {ind+1} => {np.round(pred[ind], 4)*100}%")